---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.tail(10)

490                                  Lab: B12 969 2007\n
491                                   )and 8mo in 2009\n
492    .Moved to USA in 1986. Suffered from malnutrit...
493                                              r1978\n
494    . Went to Emerson, in Newfane Alaska. Started ...
495    1979 Family Psych History: Family History of S...
496    therapist and friend died in ~2006 Parental/Ca...
497                         2008 partial thyroidectomy\n
498    sPt describes a history of sexual abuse as a c...
499    . In 1980, patient was living in Naples and de...
dtype: object

In [2]:
def completeMonthName(month):
    if month == "Jan":
        return "1"
    elif month == "Feb":
        return "2"
    elif month == "Mar":
        return "3"
    elif month == "Apr":
        return "4"
    elif month == "May":
        return "5"
    elif month == "Jun":
        return "6"
    elif month == "Jul":
        return "7"
    elif month == "Aug":
        return "8"
    elif month == "Sep":
        return "9"
    elif month == "Oct":
        return "10"
    elif month == "Nov":
        return "11"
    elif month == "Dec":
        return "12"

In [3]:
def data_preprocess(dates):
    
    # list to hold the formatted dates
    formatted_dates = []
    
    for date in (dates):
        flag = 0
        date = date.strip()
        # split the dates acc. to the joiners present
        for char in ['/', '-', ' ']:
            if date.split(char)[0] == date:
                # if char type of joiner is not present in this date
                continue
            else:
                lst = date.split(char)
                if lst[0][0].isdigit():
                    # check if the date is month preceding or date
                    if len(lst) == 3:
                        #check if day number is present in the date or not
                        if lst[1][0].isdigit():
                            #check if the middle term in the date given is month or day
                            day, month, year = lst[1], lst[0].strip('.').strip(','), lst[2]
                        else:
                            day, month, year = lst[0], lst[1].strip('.').strip(','), lst[2]

                    elif len(lst) == 2:
                        day, month, year = '1', lst[0].strip('.').strip(','), lst[1]

                else:
                    if len(lst) == 3:
                        if len(lst[1]) > 3:
                            # handles the superscript part of a day i.e th, nd, st
                            lst[1] = lst[1].strip(',')[:-2]

                        day, month, year = lst[1].strip(','), lst[0].strip('.'), lst[2]

                    elif len(lst) == 2:
                        day, month, year = '1', lst[0].strip('.'), lst[1]
                flag = 1
            
        if not flag:
            # checks if only year is provided
            day, month, year = '1', '1', date
        
        if not month[0].isdigit():
            # converts month in string to numeric
            month = completeMonthName(month[:3])
        if int(month) > 12:
            return False
        
        if len(year) == 2:
            year = '19' + year
        lst = [year, month, day]
        formatted_dates.append("/".join(lst))
    return formatted_dates

In [14]:
def date_sorter():
    
    # Your code here
    one = df.str.extract(r'((?:\d{1,2})(?:(?:/|-)\d{1,2})(?:(?:/|-)\d{2,4}))')
    
    two = df.str.extract(r'((?:\d{,2}\s)?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:-|\.|\s|,)\s?\d{,2}[a-z]*(?:-|,|\s)?\s?\d{2,4})')
    
    three = df.str.extract(r'((?:\d{1,2}(?:-|/))?(?:19|20)\d{2})')
    
    dates = one.fillna(two).fillna(three)
    
    date = list(dates)
    #preprocessing and cleaning data
    dates = data_preprocess(date)
    #data returned in the required format
    
    #(IMP) sort the date to year, month, day accordingly and returning the index from the sorted object.
    dates_index = [date[0] for date in sorted(enumerate(dates), key=lambda x: (int(x[1].split('/')[0]),int(x[1].split('/')[1]), int(x[1].split('/')[2])))]
    
    #converting list into pandas series
    dates_index = pd.Series(dates_index)
    
    return dates_index